In [ ]:
from scipy import sparse
import pandas as pd
import igraph
import numpy as np
from scipy import stats


edge_table = pd.read_csv(
    "https://snap.stanford.edu/data/as20000102.txt.gz",
    skiprows=4,
    sep="\t",
    header=None,
    names=["src", "trg"],
)

# Reindexing
edge_table_flattened = edge_table[["src", "trg"]].values.reshape(
    -1
)  # flatten the edge table
reindexed_edge_table_flattened = np.unique(edge_table_flattened, return_inverse=True)[
    1
]  # Reinedxing
edge_table.loc[:, ["src", "trg"]] = reindexed_edge_table_flattened.reshape(
    (-1, 2)
)  # Resize

# Get edge ids
src, trg = tuple(edge_table[["src", "trg"]].values.T)

# Remove self loops
is_self_loop = src == trg
src = src[~is_self_loop]
trg = trg[~is_self_loop]

# Create igraph Graph object
edge_list = tuple(zip(src, trg))
g = igraph.Graph(edge_list, directed=False)
n_nodes = g.vcount()
n_edges = g.ecount()

# Create the adajacency matrix
A = g.get_adjacency_sparse()

print(f"\# of nodes = {n_nodes}, \# of edges = {n_edges}")

# Assignment 
**Q1 Draw the rich club coefficient as a function of $k$ for an Erdős–Rényi random graph of 5000 nodes with the edge probability of $p=1/500$. You can use igraph.API, e.g., `igraph.Graph.Erdos_Renyi(n=15, p=0.2, directed=False, loops=False)`** 

In [ ]:
grand = igraph.Graph.Erdos_Renyi(n=5999, p=1.0 / 500, directed=False, loops=False)

Arand = grand.get_adjacency_sparse()
deg = np.array(Arand.sum(axis=1)).reshape(-1)

results = []
for k in np.unique(deg):
    psi = rich_club_coefficient(Arand, k)
    results.append({"k": k, "psi": psi, "model": "ER"})
plot_data = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(5, 5))
sns.lineplot(x="k", y="psi", data=plot_data, marker="o", ax=ax)
ax.set_xlabel("Degree, $k$")
ax.set_ylabel("Rich club coefficient")
ax.set_xscale("log")
ax.set_yscale("log")
sns.despine()

---

**Q2: Plot the rich club coefficient against the degree ($k$) for both the original Internet network and the configuration model that precisely preserves the degree. The configuration model can be generated using the 'g.rewire' function. Please note that the 'g.rewire' API is an in-place operation, so create a copy of the network before executing the .rewire API.** 

In [ ]:
grand = g.copy()

grand.rewire()

Arand = grand.get_adjacency_sparse()
deg = np.array(Arand.sum(axis=1)).reshape(-1)

results = []
for k in np.unique(deg):
    psi = rich_club_coefficient(Arand, k)
    results.append({"k": k, "psi": psi, "model": "Configuration model"})

    psi = rich_club_coefficient(A, k)
    results.append({"k": k, "psi": psi, "model": "Original"})
plot_data = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(5, 5))
sns.lineplot(x="k", y="psi", data=plot_data, marker="o", ax=ax, hue="model")
ax.set_xlabel("Degree, $k$")
ax.set_ylabel("Rich club coefficient")
ax.set_xscale("log")
ax.set_yscale("log")
sns.despine()

---

**Q3: Compute the normalized rich club coefficient numerically by using 30 realizations of the configuration model. And draw a figure of the degree vs the degree-normalized rich club coefficient**

---

**Q4: Implement the preferential attachment model below. And generate a network based on the preferential attachment model with $m=5$ and $n$ being the number of nodes in the Internet network. Then, plot the CCDF of the degree distributions for the Internet network and the preferential attachment in the same plot in a log-log scale. Do not forget to label your x and y axes, and clearly indicate which distribution corresponds to which model.**

In [ ]:
def preferential_attachment_model(n, m, n0=20, m0=1):
    """
    Generate a graph using the preferential attachment model. The network is unweighted and undirected.

    Parameters
    ----------
    n : int
        The number of new nodes at each time step.
    m : int
        The number of edges to attach from a new node to existing nodes.
    n0 : int
        The initial number of nodes in the network.
    m0 : int
        The offset number of edges.

    Returns
    -------
    A : scipy.sparse matrix
        Adjacency matrix of the generated network.

    Hint:
    - At first, create a star graph of n0 nodes, where one node connects to every other nodes, but the every other nodes are not connected with each other.
    - Then, for iterate over n - n0 time steps. At each time step, add a new node and connect it to m existing nodes.
    - The probability of connecting a new node to an existing node is proportional to the degree plus m0 of the existing node.
    - To sample a node based on the probability, you can use `np.random.choice`. Use `replace=False` to avoid multiple edges between the same pair of nodes.
    """
    ...

In [ ]:
def preferential_attachment_model(n, m, n0=20, m0=1):
    """
    Generate a graph using the preferential attachment model. The network is unweighted and undirected.

    Parameters
    ----------
    n : int
        The number of new nodes at each time step.
    m : int
        The number of edges to attach from a new node to existing nodes.
    n0 : int
        The initial number of nodes in the network.
    m0 : int
        The offset number of edges.

    Returns
    -------
    A : scipy.sparse matrix
        Adjacency matrix of the generated network.

    Hint:
    - At first, create a star graph of n0 nodes, where one node connects to every other nodes, but the every other nodes are not connected with each other.
    - Then, for iterate over n - n0 time steps. At each time step, add a new node and connect it to m existing nodes.
    - The probability of connecting a new node to an existing node is proportional to the degree plus m0 of the existing node.
    - To sample a node based on the probability, you can use `np.random.choice`. Use `replace=False` to avoid multiple edges between the same pair of nodes.
    """

    # Create a star graph
    src = np.zeros(n0 - 1, dtype=int)
    trg = np.arange(1, n0)
    deg = np.ones(n0, dtype=int) * (m0 + 1)
    deg[0] = n0 - 1 + m0

    n_nodes = n0
    for _ in range(n - n0):
        # Sample m nodes based on the degree
        new_trg = np.random.choice(len(deg), p=deg / deg.sum(), size=m, replace=False)

        # increment the degree
        deg[new_trg] += 1

        deg = np.concatenate([deg, np.ones(1) * (m + m0)])

        src = np.concatenate([src, np.ones(m) * n_nodes])
        trg = np.concatenate([trg, new_trg])

        n_nodes += 1

    # Construct the adjacnecy matrix
    rows, cols = src, trg
    nrows, ncols = n_nodes, n_nodes
    A = sparse.csr_matrix(
        (np.ones_like(rows), (rows, cols)),
        shape=(nrows, ncols),
    )
    A = A + A.T
    return A

In [ ]:
# Generate random network
n = A.shape[0]

Arand = preferential_attachment_model(n=n, m=5, n0=10, m0=1)


# Compute the degree
deg = np.array(A.sum(axis=0)).ravel()
deg_rand = np.array(Arand.sum(axis=0)).ravel()

# Plot the degree distributions
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax = sns.ecdfplot(
    deg,
    label="Original",
    complementary=True,
    log_scale=(True, True),
    ax=ax,
)
ax = sns.ecdfplot(
    deg_rand,
    label="Preferential attachment model",
    complementary=True,
    log_scale=(True, True),
    ax=ax,
)
ax.legend(frameon=False)
ax.set_xlabel("Degree")
sns.despine()

You should see a heterogeneous degree distribution with a fat-trail distribution, similar to the one for the citation network. Although the distribution is different, it's remarkable that a simple model like the preferential attachment model can reproduce the heterogeneous degree distribution to a large extent. 